# GeoEPIC Tutorial

- This notebook is an end-to-end tutorial for the **GeoEPIC** Python package. 
- It walks through input file preparation, single-site and batched simulations, and model **calibration workflow**.

<!-- ![Overview](./tutorial_assests/geo_epic_overview.png) -->
<div align="center">
<img src="./tutorial_assests/geo_epic_overview.png" width="600">
</div>

## 1) Prerequisites & Installation

**Requirements:**
- Windows OS
- VSCode or similar IDE (Recommended)
- Anaconda package manager

**Recommended setup :**

1. Download the setup script from: https://smarsgroup.github.io/geo_epic_win/epic_setup.bat and execute the below commands in command prompt.

    ```bat
    call epic_setup.bat
    conda activate epic_env
    ```
    The setup script will install Anaconda if it doesn't exist already, then creates a new conda environment named `epic_env` with GeoEPIC installed. 

2. Change the Python kernel of this notebook to the `epic_env` conda environment. Run the cell below to verify the installation 

In [2]:

try:
    from geoEpic.io import SOL, DLY, OPC, SIT
    from geoEpic.core import Site, EPICModel, Workspace
except Exception as e:
    print("GeoEPIC not detected.")
# from geoEpic.io import SOL, DLY, OPC, SIT
try:
    import pygmo as pg
except Exception as e:
    print("PyGMO not detected.")

# print("GeoEPIC detected.")

GeoEPIC not detected.
PyGMO not detected.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2) Create a GeoEPIC Workspace

The below command creates a sample workspace folder named `Test` with sample input files, and the EPIC executable.

```bash
geo_epic workspace -new Test
```

In [4]:
!geo_epic workspace -new Test

'geo_epic' is not recognized as an internal or external command,
operable program or batch file.


Structure of a workspace folder is as shown below:

- 📁 Test
  - 📁 model  
    - ⚙️ EPIC1102.exe  
    - 📄 CROPCOM.DAT  
    - 📄 PARM.DAT  
    - 📄 EPICRUN.DAT  
    . . . . . . 
  - 📁 opc  
  - 📁 soil  
  - 📁 site  
  - 📁 weather
  - ⚙️ config.yml  
  - 🗒️ info.csv  


In [ ]:
from pathlib import Path
workspace_dir = Path('./Test')

## 3) Input files Preparation

`geoEpic.sptial` module provides helpers to build EPIC input files from spatial datasets:
 - Soil Data: [`USDA-SSURGO`](https://www.nrcs.usda.gov/resources/data-and-reports/soil-survey-geographic-database-ssurgo), [`ISIRC-SoilGrids`](https://soilgrids.org/)
 - Daily Weather: [`Daymet`](https://daymet.ornl.gov/), [`AgEra5`](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-agrometeorological-indicators)
 - Elevation, Slope: [`GLO-30`](https://registry.opendata.aws/copernicus-dem/), [`SRTM`](https://www2.jpl.nasa.gov/srtm/)

These examples require internet access and proper credentials setup.

### Study Region: Mead, Nebraska
<!-- <div align="center"> 08/08/2023 -->
<div style="display: flex; gap: 20px;">
   <div style="width: 30%; text-align: center;">
    <img src="./tutorial_assests/fileds_NE_18ft.png" style="width: 100%;">
    <p>Mead, Nebraska (Landsat-8)</p>
  </div>
  <div style="width: 30%; text-align: center;">
    <img src="./tutorial_assests/USDA_NASS_CDL.png" style="width: 100%;">
    <p>
      USDA NASS Cropland Data Layer (2023)<br>
      <span style="display: inline-block; width: 12px; height: 12px; background-color: yellow; border: 1px solid #ccc; margin-right: 5px;"></span> Corn  
      <span style="display: inline-block; width: 12px; height: 12px; background-color: green; border: 1px solid #ccc; margin-left: 15px; margin-right: 5px;"></span> Soybean
    </p>
  </div>
  <div style="width: 30%; text-align: center;">
    <img src="./tutorial_assests/your_third_image.png" style="width: 100%;">
    <p>📊 [Your Third Image Description Here]</p>
  </div>
</div>
<!-- </div> -->

In [ ]:
# location of NE-2 site
lat, lon = 41.1649, -96.4701

#####  3.1) Creating Soil files

In [ ]:
from geoEpic.spatial import SSURGO, SoilGrids
from geoEpic.io import SOL

#fetch soil data from USDA SSURGO
soil_ssurgo = SSURGO.fetch(lat=lat, lon=lon)
# soil_grids = SoilGrids.fetch(lat=lat, lon=lon) #ISIRC soilgrids
soil_ssurgo.save(workspace_dir/'soil'/'ssurgo.SOL')

#load an existing SOL file 
soil = SOL.load(workspace_dir/'soil'/'ssurgo.SOL')
soil_ssurgo.layers_df.head()

#####  3.2) Weather files

In [ ]:
from geoEpic.spatial import Daymet, AgEra5
from geoEpic.io import DLY

dly_daymet = Daymet.fetch(lat=lat, lon=lon)
# dly_era5 = AgEra5.fetch(lat=lat, lon=lon)
dly_daymet.save(workspace_dir/'weather'/'daily'/'daymt.DLY')
dly_daymet.head()

#####  3.3) SIT files

In [ ]:
from geoEpic.spatial import DEM
from geoEpic.io import SIT
elevation, slope = DEM.fetch(lat=lat, lon=lon)

#### 3.4) Editing Crop Management files

The crop management file contains information about management practices carried out at the specific site. <br> Each row represents a specific management operation, including crop planting, fertilizer applications, irrigation scheduling, harvesting, and tillage practices. <br> The structure and valid entries for `OPC` files are detailed in the [`EPIC1102` user manual](./tutorial_assests/epic1102-user-manual.pdf#page=69).

In [ ]:
from geoEpic.io import OPC
opc = OPC.load('./umstead.OPC')
opc.head(20)

In [ ]:
# Select auto irrigation implement ID
opc.IAUI = 72
# Apply fertilizer 
fertilizer = {'opID':71, 'cropID':2, 'fertID':52,
              'date': '2015-04-01', 'OPV1': 160}
opc.update(fertilizer)
# Remove certain operation(s)
opc.remove(opID=71, date='2015-04-01')
opc.head(20)

In [ ]:
# Save the OPC file with changes
opc.save('umstead.OPC')

## 6) Single-site simulation (`Site`, `EPICModel`)

In [ ]:
from geoEpic.core import Site, EPICModel

# create a site object
site = Site(
    site_id = 'Ne2',
    opc=str(workspace_dir/'opc'/'site.OPC'),
    dly=str(workspace_dir/'weather'/'site.DLY'),
    sol=str(workspace_dir/'soil'/'site.SOL'),
    sit=str(workspace_dir/'sites'/'site.SIT'),
)

In [ ]:
Path(workspace_dir/'outputs').mkdir(exist_ok=True, parents=True)

# Execute the model at this particular site
model = EPICModel(workspace_dir/'model'/'EPIC1102.exe')
model.start_date = '2015-01-01'
model.duration = 5 # years
model.output_types = ['ACY', 'DGN']
model.output_dir = workspace_dir/'outputs'
model.run(site)
model.close()

NameError: name 'Path' is not defined

#### Reading outputs (`ACY`, `DGN`)

In [ ]:
from geoEpic.io import ACY, DGN, DWC

yields = ACY(site).get_var('YLDG')

In [ ]:
lai = DGN(site).get_var('LAI')

plt.figure(figsize=(4, 3))
plt.plot(lai['LAI'], marker='o')
plt.title("Leaf Area Index")
plt.xlabel("Day")
plt.ylabel("LAI")
plt.show()

<!-- <div align="center"> -->
 <img src="./tutorial_assests/corn.png" width="60%">
<!-- </div> -->

## 8) Managing multiple sites with `Workspace` class

The Workspace class is a powerful component of `GeoEPIC` that enables efficient management and execution of `EPIC` simulations across multiple sites, significantly streamlining the workflow for regional-scale agricultural modeling studies. <br>
Workspace object is initialized by two key configuration elements: <br>
(1) configuration file that specifies global simulation settings <br>
(2) run_info file containing metadata on individual simulation sites. 

In [ ]:
#initialize Workspace object
exp = Workspace(workspace_dir/'config.yml')

#clear previous logs and outputs
exp.clear_logs()
exp.clear_outputs()

In [ ]:
# run the simulations for all the sites
exp.run()

In [ ]:
Path(workspace_dir/'lai_export').mkdir(exist_ok=True, parents=True)
 
# save Yield data to csv files for each site
@exp.routine
def save_lai(site):
    lai = DGN(site).get_var('LAI')
    lai.to_csv(workspace_dir/f'lai_export/{site.site_id}.csv')
    

# calculate average annual ET for soybean and log them
#
@exp.logger
def avg_annual_et(site):
    daily_et = DWC(site).get_var('ET')
    filtered_data = daily_et[(daily_et['ET'] > 0.05) 
                           & (daily_et['Y'].between(2017, 2020)) 
                           & (daily_et['M'].between(5, 9))]
    # sum ET by year
    annual_et = filtered_data.groupby('Y')['ET'].sum().reset_index()
    # get average ET for soybean
    soyb_data = annual_et[annual_et['CPNM'] == 'SOYB']
    avg_et_soyb = soyb_data['ET'].mean() if not soyb_data.empty else None
    return {'SOYB_ET': avg_et_soyb}

In [ ]:
exp.run()

#### Spatial plot of Average annual Evapotranspiration

In [ ]:
from tutorial_assets import spatial_plot

logged_et = exp.fetch_log('avg_annual_et')
spatial_plot('Mead_NE.shp', logged_et)

<!-- <div align="center"> -->
 <img src="./tutorial_assests/ET_annual_soybean.png" width="60%">
<!-- </div> -->

## 9) Calibration WorkFlow

The calibration setup involves defining a fitness criterion that quantifies the difference between simulated outputs and observed data and selecting the parameters that significantly influence the required outputs. The figure shows the overview of Calibration iterations in GeoEPIC.

<!-- <div align="center"> -->
 <img src="./tutorial_assests/model_calibration.png" width="60%">
<!-- </div> -->

The steps involved in the setup are shown below:
1. Define a logger to compute per-site error(s) after each simulation.
2. Define an objective to aggregate logged metrics.
3. Mark sensitive parameters in `CropCom`/`Parm`, then build a PyGMO problem via `exp.make_problem(...)`.
4. Choose an algorithm (e.g., PSO) and optimize.

In [ ]:
exp.config['select'] = 'ET == 1'

@exp.logger
def et_error(site):
    # Get daily ET from EPIC simulation
    daily_et = DWC(site).get_var('ET')
    filtered_data = daily_et[(daily_et['ET'] > 0.05) 
                           & (daily_et['Y'].between(2017, 2020)) 
                           & (daily_et['M'].between(5, 9))]
    
    # Load target ET data for this site
    target_et = pd.read_csv(f'./target_et/{site.site_id}.csv')
    target_et['date'] = pd.to_datetime(target_et['date'])
    # Merge on date (intersection)
    merged_data = pd.merge(filtered_data, target_et, on='date', how='inner')
    # Calculate year-wise error for soybean crop
    soyb_data = merged_data[merged_data['CPNM'] == 'SOYB']
    if soyb_data.empty: return {'error': None}
    # Group by year and sum ET values
    sim_annual = soyb_data.groupby('Y')['ET'].sum()  # simulation ET
    obs_annual = soyb_data.groupby('Y')['ET_obs'].sum()  # observed ET
    # Calculate year-wise RMSE using numpy
    year_rmse = np.sqrt(np.mean((sim_annual - obs_annual) ** 2))
    return {'error': year_rmse}


@exp.objective
def aggregate():
    logged = exp.fetch_log('et_error').dropna()
    return [logged['error'].mean()]


exp.run()

### Compare simulated and observed ET

In [3]:
from tutorial_assets import compare_et
compare_et()

ModuleNotFoundError: No module named 'tutorial_assets'

#### Setting sensitive parameters

In [ ]:
from geoEpic.core import CropCom, Parm

cropcom = CropCom(exp.model.path)
cropcom.set_sensitive(['WA','HI','WSYF'], [1])
cropcom.vars

In [ ]:
parm = Parm(exp.model.path)
parm.set_sensitive(['PARM28','PARM68'])
parm.vars

#### Create a optimization problem to interface with pygmo library 

In [ ]:
problem = exp.make_problem(cropcom, parm)


Refer to the PyGMO [documentation](https://esa.github.io/pygmo2/algorithms.html) to experiment with different algorithms and customizable parameters to find the most effective optimization strategy.

In [ ]:
import pygmo as pg
# Initialize the problem with PSO algorithm
problem.init(algorithm = pg.pso_gen, memory=True)
problem.optimize(population_size = 30, generations = 10)

<!-- <code> -->
<div style="font-family:'Consolas','Courier New',monospace;color:rgb(60, 60, 60)">
Fitness before optimization: [3.0133333]<br>
Setting Initial Population<br>
optimizing...
</div>
<div style="font-family:Consolas,'Courier New',monospace;color:rgb(60,60,60)">
  100%|██████████| 10/10 [11:10<00:00, 67.05s/gen, best_fitness=0.776]<br>
  Final best fitness: [0.776]
</div>

## 10) Troubleshooting & Tips

- Check `log/` for errors during model execution;
- Start with a small subset of sites: `select: Random(0.05)`.
- Edit `.sens` file in the model folder to set valid range for parameters
- Begin calibration with a few sensitive parameters; expand gradually.
- Edit `exp.num_of_workers` to expand or constrain the no of parallel simulations.